# Experiments for hash-based PRNGs

In [1]:
from ctypes import *
from ctypes.util import find_library

In [3]:
libname = 'tomcrypt'
libpath = f'/opt/libtom/lib/lib{libname}.dylib'
LTC = cdll.LoadLibrary(libpath)
LTC

<CDLL '/opt/libtom/lib/libtomcrypt.dylib', handle 7ff9dd7eb310 at 0x11009a160>

In [4]:
#---------------------------------------------------------------
# get list of all supported constants followed by a list of all
# supported sizes.  One alternative: these lists may be parsed
# and used as needed.
print('  all supported constants and their values:')

# get size to allocate for constants output list
str_len = c_int(0)
ret = LTC.crypt_list_all_constants(None, byref(str_len))
print('    need to allocate %d bytes \n' % str_len.value)

# allocate that size and get (name, size) pairs, each pair
# separated by a newline char.
names_sizes = c_buffer(str_len.value)
ret = LTC.crypt_list_all_constants(names_sizes, byref(str_len))
print(names_sizes.value.decode())

  all supported constants and their values:
    need to allocate 627 bytes 

PK_PUBLIC,0
PK_PRIVATE,1
PKA_RSA,0
PKA_DSA,1
LTC_PKCS_1,1
LTC_PKCS_1_EMSA,1
LTC_PKCS_1_EME,2
LTC_PKCS_1_V1_5,1
LTC_PKCS_1_OAEP,2
LTC_PKCS_1_PSS,3
LTC_MRSA,1
MIN_RSA_SIZE,1024
MAX_RSA_SIZE,4096
LTC_MKAT,0
LTC_MECC,1
ECC_BUF_SIZE,256
ECC_MAXSIZE,66
LTC_MDSA,1
LTC_MDSA_DELTA,512
LTC_MDSA_MAX_GROUP,512
LTC_DER_MAX_PUBKEY_SIZE,4096
LTC_MILLER_RABIN_REPS,35
LTC_CTR_MODE,1
CTR_COUNTER_LITTLE_ENDIAN,0
CTR_COUNTER_BIG_ENDIAN,4096
LTC_CTR_RFC3686,8192
MAXBLOCKSIZE,128
TAB_SIZE,32
ARGTYPE,0
LTM_DESC,1
TFM_DESC,0
GMP_DESC,0
LTC_FAST,1
LTC_NO_FILE,0
ENDIAN_LITTLE,1
ENDIAN_BIG,0
ENDIAN_32BITWORD,0
ENDIAN_64BITWORD,1
ENDIAN_NEUTRAL,0


In [93]:
# print selected sizes
print('\n  selected sizes:')

names = [b'sha256_state', b'sha512_state']
for name in names:
    size = c_int(0)
    rc = LTC.crypt_get_size(name, byref(size))
    value = size.value
    print('    %-25s  %d' % (name.decode(), value))


  selected sizes:
    sha256_state               112
    sha512_state               208


In [152]:
#---------------------------------------------------------------
# here is an example of how a wrapper can make Python access
# more Pythonic

def _get_size(name):
    size = c_int(0)
    rc = LTC.crypt_get_size(name, byref(size))
    return size.value

sha256_state_struct_size = _get_size(b'sha256_state')
sha512_state_struct_size = _get_size(b'sha512_state')

class SHAWalker():
    def __init__(self, seed:int=0):
        self.state = create_string_buffer(sha256_state_struct_size)
        self.md = create_string_buffer(32)
        LTC.sha256_init(self.state)
        if seed>0:
            self.jump(seed)
        
    def step(self):
        LTC.sha256_process(self.state, b'0', 1)

    def jump(self, nsteps: int):
        LTC.sha256_process(self.state, b'0' * nsteps, nsteps)
    
    def update(self, data):
        if not isinstance(data, bytes):
            raise TypeError("Unicode-objects must be encoded before hashing - tomcrypt")
        LTC.sha256_process(self.state, data, len(data))
        
    def digest(self):
        LTC.sha256_done(self.state, self.md)
        return self.md.value

    def hexdigest(self):
        return self.digest().hex()

In [153]:
sw = SHAWalker()
sw.hexdigest()

'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855'

In [154]:
import hashlib

spy = hashlib.sha256()
spy.hexdigest()

'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855'

In [155]:
class SHAWalkerPy():
    def __init__(self, seed:int=0):
        self.s256 = hashlib.sha256()
        if seed>0:
            self.jump(seed)
        
    def step(self):
        self.s256.update(b'0')

    def jump(self, nsteps: int):
        self.s256.update(b'0' * nsteps)
        
    def digest(self):
        return self.s256.digest()
    
    def update(self, data):
        return self.s256.update(data)

    def hexdigest(self):
        return self.s256.hexdigest()

In [156]:
swtl = SHAWalker()
swpy = SHAWalkerPy()
print(f"{swtl.hexdigest()}\n{swpy.hexdigest()}")

e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855
e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855


In [157]:
data = b'abc'
swtl.update(data)
swpy.update(data)
print(f"{swtl.hexdigest()}\n{swpy.hexdigest()}")

56ec7a7ce28a186dbc0913330ad45b9dea61d814ec8829aba79cc77c9353a8ee
ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad


In [163]:
swtl.step()
swpy.step()
print(f"{swtl.hexdigest()}\n{swpy.hexdigest()}")

3db907a741548444bd90a57c270433f9caa52844d36693f92ddc718cf7182fdf
277d07fce47a8c7791ae69093f708092e235116f6d2bf6ec3e06c372b057617a


## Timings

In [103]:
nsteps = 500
swt = SHAWalker()
swp = SHAWalkerPy()

In [104]:
%%timeit -n 1000 -r 10
for i in range(nsteps):
    swt.step()
    swt.digest()

1.05 ms ± 12.4 µs per loop (mean ± std. dev. of 10 runs, 1000 loops each)


In [90]:
%%timeit -n 1000 -r 10
for i in range(nsteps):
    swp.step()
    swp.digest()

504 µs ± 10.7 µs per loop (mean ± std. dev. of 10 runs, 1000 loops each)


Important note: I tried to implement a quick test of a next/jump API, and it's *twice* as slow as the Python one. This was doing it in the easiest way possible, via ctypes, but it's a good indicator that cutting this overhead will require either Cython or pure C/C++.  

More importantly, as [stated in the companion notebook](libtomcrypt-ctypes.ipynb#WARNING:-this-SHA256-object-is-broken!), not only is it slower, it's also wrong (for reasons I explain there).